In [1]:
import pandas as pd
import numpy as np
import unidecode
import Levenshtein

In [2]:
df_tratado = pd.read_excel('../../dados_finais_2024/final_data.xlsx', sheet_name='Súmulas')
df_nomes = pd.read_csv('../../scrapers/cleaned_data_2024/final_data_regata.csv', low_memory=False)

In [3]:
df_tratado.columns

Index(['ID Resultado', 'ID Competidor', 'Nome Competidor', 'ID Competição',
       'Classe Vela', 'Pontuação Regata', 'Descarte', 'Flotilha',
       'Posição Geral', 'Punição', 'Pontuação Total', 'Nett',
       'Nome Competição', 'Regata'],
      dtype='object')

In [4]:
df_tratado.head(2)

,ID Resultado,ID Competidor,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Descarte,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição,Regata
0,1,1,"WILLIAM JONES, EVAN DEPAUL",17.0,49ER,11,1,GERAL,1,UFD,44.0,33.0,World Championship 2017,NaN
1,2,2,"DOMINIK BUKSAK, SZYMON WIERZBICKI",7.0,49ER,3,0,GERAL,1,NaN,89.0,66.0,Semaine Olympique Francaise De Voile 2022,NaN


In [5]:
df_nomes.columns

Index(['ID Resultado', 'ID Competidor', 'Nome Competidor', 'ID Competição',
       'Classe Vela', 'Pontuação Regata', 'Descarte', 'Flotilha',
       'Posição Geral', 'Punição', 'Pontuação Total', 'Nett',
       'Nome Competição', 'Regata', 'Nacionalidade'],
      dtype='object')

In [6]:
df_nomes.head(2)

,ID Resultado,ID Competidor,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Descarte,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição,Regata,Nacionalidade
0,1,3502,MINA MOBEKK,19,IQFOIL FEM.,15.0,0,NaN,1,0,207,116,European Championship 2023,1Q,NOR
1,2,3502,MINA MOBEKK,19,IQFOIL FEM.,2.0,0,NaN,1,0,207,116,European Championship 2023,2Q,NOR


In [7]:
df_nomes['Nome Competidor'] = df_nomes['Nome Competidor'].apply(lambda x: unidecode.unidecode(x).upper())
df_tratado['Nome Competidor'] = df_tratado['Nome Competidor'].apply(lambda x: unidecode.unidecode(x).upper())

In [8]:
competicoes_tratadas = df_tratado['Nome Competição'].unique()
competicoes_sujas = df_nomes['Nome Competição'].unique()
for competicao in competicoes_sujas:
    if competicao not in competicoes_tratadas:
        print(competicao)

European Championship 2021 - Gold Fleet & Medal Race
European Championship 2021 - Silver Fleet
European Championship 2023 - Bronze Fleet
European Championship 2023 - Gold Fleet
European Championship 2023 - Silver Fleet


In [9]:
for competicao in df_nomes['Nome Competição'].unique():
    if competicao not in df_tratado['Nome Competição'].unique():
        df_nomes[df_nomes['Nome Competição'] == competicao] = str(competicao).split(' - ')[0].strip()

C:\Users\silla\AppData\Local\Temp\ipykernel_19732\1998865097.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'European Championship 2021' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_nomes[df_nomes['Nome Competição'] == competicao] = str(competicao).split(' - ')[0].strip()
C:\Users\silla\AppData\Local\Temp\ipykernel_19732\1998865097.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'European Championship 2021' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_nomes[df_nomes['Nome Competição'] == competicao] = str(competicao).split(' - ')[0].strip()
C:\Users\silla\AppData\Local\Temp\ipykernel_19732\1998865097.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of panda

In [10]:
competicoes_tratadas = df_tratado['Nome Competição'].unique()
competicoes_sujas = df_nomes['Nome Competição'].unique()
for competicao in competicoes_sujas:
    if competicao not in competicoes_tratadas:
        print(competicao)

In [11]:
nomes_tratados = pd.DataFrame()
nomes_tratados['Concat'] = pd.unique(df_tratado['Nome Competidor'] + " % " + df_tratado['Nome Competição'])
nomes_tratados['Nome Competidor'] = nomes_tratados['Concat'].apply(lambda x: x.split('%')[0].strip())
nomes_tratados['Nome Competição'] = nomes_tratados['Concat'].apply(lambda x: x.split('%')[1].strip())
nomes_tratados.drop(columns=['Concat'], inplace=True)

In [12]:
nomes_sujos = pd.DataFrame()
nomes_sujos['Concat'] = pd.unique(df_nomes['Nome Competidor'] + " % " + df_nomes['Nome Competição'])
nomes_sujos['Nome Competidor'] = nomes_sujos['Concat'].apply(lambda x: x.split('%')[0].strip())
nomes_sujos['Nome Competição'] = nomes_sujos['Concat'].apply(lambda x: x.split('%')[1].strip())
nomes_sujos.drop(columns=['Concat'], inplace=True)

In [13]:
nomes_sujos['Match'] = ''
nomes_sujos['Score'] = 1000
for i, sujo in nomes_sujos.iterrows():
    competicao = sujo['Nome Competição']
    nome = sujo['Nome Competidor']
    nomes_1 = nome.split(' ')

    best_score = 1000
    best_match = nome

    for name_ in nomes_tratados[nomes_tratados['Nome Competição'] == competicao]['Nome Competidor']:
        names_2 = name_.split(' ')
        score = 0

        for n1 in nomes_1:
            best = 1000
            for n2 in names_2:
                score_ = Levenshtein.distance(n1, n2)
                if score_ < best:
                    best = score_
            score += best

        if score < best_score:
            best_score = score
            best_match = name_
    
    nomes_sujos.at[i, 'Match'] = best_match
    nomes_sujos.at[i, 'Score'] = best_score

In [14]:
nomes_sujos[nomes_sujos['Score'] < 6]

,Nome Competidor,Nome Competição,Match,Score
0,MINA MOBEKK,European Championship 2023,MINA MOBEKK,0
1,SHARON KANTOR,European Championship 2023,SHARON KANTOR,0
2,EMMA WILSON,European Championship 2023,EMMA WILSON,0
3,HELENE NOESMOEN,European Championship 2023,HELENE NOESMOEN,0
4,SHAHAR TIBI,European Championship 2023,SHAHAR TIBI,0
...,...,...,...,...
9271,AIJA AMBRASA,World Championship 2024,AIJA AMBRASA,0
9272,SOPHIE RENNIE,World Championship 2024,SOPHIE RENNIE,0
9273,EMILIA KOSTI,World Championship 2024,EMILIA KOSTI,0
9274,MARTYNA DAKOWICZ,World Championship 2024,MARTYNA DAKOWICZ,0


In [15]:
nomes_sujos

,Nome Competidor,Nome Competição,Match,Score
0,MINA MOBEKK,European Championship 2023,MINA MOBEKK,0
1,SHARON KANTOR,European Championship 2023,SHARON KANTOR,0
2,EMMA WILSON,European Championship 2023,EMMA WILSON,0
3,HELENE NOESMOEN,European Championship 2023,HELENE NOESMOEN,0
4,SHAHAR TIBI,European Championship 2023,SHAHAR TIBI,0
...,...,...,...,...
9271,AIJA AMBRASA,World Championship 2024,AIJA AMBRASA,0
9272,SOPHIE RENNIE,World Championship 2024,SOPHIE RENNIE,0
9273,EMILIA KOSTI,World Championship 2024,EMILIA KOSTI,0
9274,MARTYNA DAKOWICZ,World Championship 2024,MARTYNA DAKOWICZ,0


In [16]:
competicoes = nomes_sujos['Nome Competição'].unique()
print(len(competicoes))

64


In [17]:
# 3 Trofeo S.A.R Princesa Sofia 2023
# 4 Semaine Olympique Francaise De Voile 2015
# 5 Semaine Olympique Francaise De Voile 2016
# 6 Semaine Olympique Francaise De Voile 2023
# 9 European Championship 2021
# 12 World Championship 2021


In [18]:
nomes_sujos[nomes_sujos['Nome Competição'] == competicoes[13]].sort_values(by='Score', ascending=False).head(20)

,Nome Competidor,Nome Competição,Match,Score
2638,"NATACHA VIOLET SAOUM, A-PEDERSEN, MATHIAS BRUU...",World Championship 2022,"NATACHA VIOLET SAOUMA-PEDERSEN, MATHIAS BRUUN ...",10
2527,"ISAAC , MCHARDIE, WILLIAM , MCKENZIE",World Championship 2022,"ISAAC MCHARDIE, WILLIAM MCKENZIE",10
2573,"JOAKIM SALSKO, V-IVERSEN, MARKUS OLIVER NIELSEN",World Championship 2022,"JOAKIM SALSKOV-IVERSEN, MARKUS OLIVER NIELSEN",10
2618,"NADINE BOHM, LENA MARIE WE, ISSKICHEL",World Championship 2022,"NADINE BOEHM, LENA WEISSKICHEL",9
2602,"GEORGIA LEWI, N-LAFRANCE, ANTONIA LEWIN-LAFRANCE",World Championship 2022,"ANTONIA LEWIN LAFRANCE, GEORGIA LEWIN LAFRANCE",9
2594,"PATRICIA SUAREZ, MARIA CANTERO IZQUIERDO",World Championship 2022,"PATRICIA SUAREZ, MARIA CANTERO",6
2567,"MARCO ANESSI PE, EDOARDO GAMBA",World Championship 2022,"MARCO ANESSI, EDOARDO GAMBA",6
5477,JUAN IGNACIO PEREZ SOLTERO,World Championship 2022,JUAN IGNACIO PEREZ,6
2536,"MAX STINGELE, LINOV SCHEEL",World Championship 2022,"MAXIMILIAN STINGELE, LINOV SCHEEL",5
5383,GIOVANNI COCCOLUTO GIORGETTI,World Championship 2022,GIOVANNI COCCOLUTO,5


In [19]:
sumulas = pd.read_csv('../../scrapers/cleaned_data_2024/final_data_regata.csv', low_memory=False)
sumulas

,ID Resultado,ID Competidor,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Descarte,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição,Regata,Nacionalidade
0,1,3502,MINA MOBEKK,19,IQFOIL FEM.,15.0,0,NaN,1,0,207,116,European Championship 2023,1Q,NOR
1,2,3502,MINA MOBEKK,19,IQFOIL FEM.,2.0,0,NaN,1,0,207,116,European Championship 2023,2Q,NOR
2,3,3502,MINA MOBEKK,19,IQFOIL FEM.,6.0,0,NaN,1,0,207,116,European Championship 2023,3Q,NOR
3,4,3502,MINA MOBEKK,19,IQFOIL FEM.,3.0,0,NaN,1,0,207,116,European Championship 2023,4Q,NOR
4,5,3502,MINA MOBEKK,19,IQFOIL FEM.,3.0,0,NaN,1,0,207,116,European Championship 2023,5Q,NOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139082,139083,51,AIJA AMBRASA,69,KITE FEM.,27.0,0,Silver,22,DNC,412.0,333.0,World Championship 2024,F8,LAT
139083,139084,4549,SOPHIE RENNIE,69,KITE FEM.,19.0,0,Silver,23,0,412.0,336.0,World Championship 2024,F8,GBR
139084,139085,1289,EMILIA KOSTI,69,KITE FEM.,27.0,0,Silver,24,UFD,441.0,360.0,World Championship 2024,F8,GRE
139085,139086,3256,MARTYNA DAKOWICZ,69,KITE FEM.,27.0,0,Silver,25,UFD,450.0,369.0,World Championship 2024,F8,CAN


In [20]:
for competicao in sumulas['Nome Competição'].unique():
    if competicao not in df_tratado['Nome Competição'].unique():
        sumulas[sumulas['Nome Competição'] == competicao] = str(competicao).split(' - ')[0].strip()

C:\Users\silla\AppData\Local\Temp\ipykernel_19732\2138070012.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'European Championship 2021' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sumulas[sumulas['Nome Competição'] == competicao] = str(competicao).split(' - ')[0].strip()
C:\Users\silla\AppData\Local\Temp\ipykernel_19732\2138070012.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'European Championship 2021' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sumulas[sumulas['Nome Competição'] == competicao] = str(competicao).split(' - ')[0].strip()
C:\Users\silla\AppData\Local\Temp\ipykernel_19732\2138070012.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. V

In [21]:
cout = 0
for i, row in sumulas.iterrows():
    competicao = row['Nome Competição']
    nome = row['Nome Competidor']

    matching_rows = nomes_sujos[nomes_sujos['Nome Competidor'] == nome]
    if not matching_rows.empty:
        sumulas.at[i, 'Nome Competidor'] = matching_rows['Match'].values[0]
    else:
        print(competicao, nome)
        cout += 1
print(cout)

European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2023  AGUILAR CHAVEZ PEON
European Championship 2022  AGUILAR CHAVEZ PEON
European Championship 2022  AGUILAR CHAV

In [22]:
sumulas

,ID Resultado,ID Competidor,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Descarte,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição,Regata,Nacionalidade
0,1,3502,MINA MOBEKK,19,IQFOIL FEM.,15.0,0,NaN,1,0,207,116,European Championship 2023,1Q,NOR
1,2,3502,MINA MOBEKK,19,IQFOIL FEM.,2.0,0,NaN,1,0,207,116,European Championship 2023,2Q,NOR
2,3,3502,MINA MOBEKK,19,IQFOIL FEM.,6.0,0,NaN,1,0,207,116,European Championship 2023,3Q,NOR
3,4,3502,MINA MOBEKK,19,IQFOIL FEM.,3.0,0,NaN,1,0,207,116,European Championship 2023,4Q,NOR
4,5,3502,MINA MOBEKK,19,IQFOIL FEM.,3.0,0,NaN,1,0,207,116,European Championship 2023,5Q,NOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139082,139083,51,AIJA AMBRASA,69,KITE FEM.,27.0,0,Silver,22,DNC,412.0,333.0,World Championship 2024,F8,LAT
139083,139084,4549,SOPHIE RENNIE,69,KITE FEM.,19.0,0,Silver,23,0,412.0,336.0,World Championship 2024,F8,GBR
139084,139085,1289,EMILIA KOSTI,69,KITE FEM.,27.0,0,Silver,24,UFD,441.0,360.0,World Championship 2024,F8,GRE
139085,139086,3256,MARTYNA DAKOWICZ,69,KITE FEM.,27.0,0,Silver,25,UFD,450.0,369.0,World Championship 2024,F8,CAN


In [23]:
sumulas.to_csv('../../scrapers/cleaned_data_2024/final_data_regata_tratado.csv', index=False)